In [17]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder

In [3]:
train_df = pd.read_csv(Path('Resources/2019loans.csv'))
test_df = pd.read_csv(Path('Resources/2020Q1loans.csv'))


In [4]:
train_df = train_df.dropna(axis='columns', how='all')
train_df = train_df.dropna()
train_df.drop(["Unnamed: 0", "index"], axis = 1, inplace = True)


train_df['int_rate'].replace('%', '')
train_df['int_rate'].astype('float') / 100
# train_df.head()
train_df.dtypes

loan_amnt                     float64
int_rate                      float64
installment                   float64
home_ownership                 object
annual_inc                    float64
                               ...   
total_bal_ex_mort             float64
total_bc_limit                float64
total_il_high_credit_limit    float64
hardship_flag                  object
debt_settlement_flag           object
Length: 84, dtype: object

In [11]:
# Convert categorical data to numeric and separate target feature for training data


train_df.loan_status.value_counts()

# Converting categorical to numeric
train_df.loan_status[(train_df['loan_status'] == 'high_risk')]=1
train_df.loan_status[(train_df['loan_status'] == 'low_risk')]=0
# train_df.loan_status.astype(float)
test_df.loan_status[(test_df['loan_status'] == 'high_risk')]=1
test_df.loan_status[(test_df['loan_status'] == 'low_risk')]=0
# test_df.loan_status.astype(float)

# seperate target feature for training data
X_train = pd.get_dummies(train_df, columns=['home_ownership', 'verification_status', 'pymnt_plan',
                                            'initial_list_status', 'application_type', 'hardship_flag',
                                            'debt_settlement_flag']).drop("loan_status", axis=1)
# y_train = train_df["loan_status"]
X_test = pd.get_dummies(train_df, columns=['home_ownership', 'verification_status', 'pymnt_plan',
                                            'initial_list_status', 'application_type', 'hardship_flag',
                                            'debt_settlement_flag']).drop("loan_status", axis=1)
                                            



<ipython-input-11-a8b79eb30a82>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.loan_status[(train_df['loan_status'] == 'high_risk')]=1
<ipython-input-11-a8b79eb30a82>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.loan_status[(train_df['loan_status'] == 'low_risk')]=0
<ipython-input-11-a8b79eb30a82>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.loan_status[(test_df['loan_status'] == 'high_risk')]=1
<ip

In [12]:
# Convert categorical data to numeric and separate target feature for testing data
X_train.head()
X_test.head()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,verification_status_Verified,pymnt_plan_n,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App,hardship_flag_N,hardship_flag_Y,debt_settlement_flag_N,debt_settlement_flag_Y
0,13375.0,0.1797,483.34,223000.0,29.99,0.0,0.0,15.0,0.0,39728.0,...,0,1,0,1,1,0,1,0,1,0
1,21000.0,0.1308,478.68,123000.0,11.26,2.0,0.0,16.0,0.0,9585.0,...,0,1,0,1,1,0,1,0,1,0
2,20000.0,0.1240,448.95,197000.0,11.28,0.0,0.0,12.0,0.0,16708.0,...,0,1,0,1,1,0,1,0,1,0
3,3000.0,0.1240,100.22,45000.0,18.08,0.0,0.0,12.0,1.0,8809.0,...,0,1,0,1,1,0,1,0,1,0
4,30000.0,0.1612,1056.49,133000.0,27.77,0.0,2.0,13.0,0.0,65420.0,...,0,1,0,1,1,0,1,0,1,0


In [ ]:
# add missing dummy variables to testing set

In [13]:
# Train the Logistic Regression model on the unscaled data and print the model score
X_train = train_df.drop("loan_status", axis=1).copy()
X_test = test_df.drop("loan_status", axis=1).copy()
y_train = train_df["loan_status"].copy()
y_test = test_df["loan_status"].copy()

In [15]:
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)

In [18]:
cols_to_add = set(X_train.columns) ^ set(X_test.columns)
for col in cols_to_add:
    X_test[col] = 0 

X_test = X_test[X_train.columns]

target_encoder = LabelEncoder().fit(y_train)
y_train = target_encoder.transform(y_train)
y_test = target_encoder.transform(y_test)

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

lr = LogisticRegression()
lr.fit(X_train, y_train)
print(lr.score(X_test, y_test))
print(lr.score(X_train, y_train))

0.5168013611229264
0.6508210180623973


C:\Users\Riley\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [20]:
# Train a Random Forest Classifier model and print the model score
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
print(rf.score(X_test, y_test))
print(rf.score(X_train, y_train))

0.635899617184177
1.0


In [21]:
# Scale the data
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [22]:
# Train the Logistic Regression model on the scaled data and print the model score
lr.fit(X_train_scaled, y_train)
print(lr.score(X_test_scaled, y_test))
print(lr.score(X_train_scaled, y_train))

0.767333049766057
0.7078817733990148


C:\Users\Riley\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [23]:
# Train a Random Forest Classifier model on the scaled data and print the model score
rf.fit(X_train_scaled, y_train)
print(rf.score(X_test_scaled, y_test))
print(rf.score(X_train_scaled, y_train))

0.6316461080391322
1.0
